# **Comprehensive Visualization of Our CNN Model**

In this section, we'll demonstrate how to **visualize our CNN's** architecture and internal details in Python. We'll use:

- **Model Summary**: to see a text-based breakdown of each layer, the output shape, and the number of parameters.
- **Graph Visualization** (via [**torchviz**](https://github.com/szagoruyko/pytorchviz)): to generate a graphical representation of how data flows through the CNN.

## **Requirements**
1. `pip install torchviz`
2. `pip install torchsummary` or `pip install torch-summary` (depending on your preference)

---


## **1. Import and Define the Model (Review)**

We'll re-use the same CNN model we built previously for classifying letters (O, X, T, S) in a 10x10 image. If you've already defined your model in the same notebook, you can skip re-defining it here. Otherwise, let's re-define it below for clarity.

In [1]:
!pip install torchviz

In [2]:
!pip install torchsummary

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchviz import make_dot

class MultiLetterCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(MultiLetterCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc = nn.Linear(12 * 1 * 1, num_classes)

    def forward(self, x):
        x = self.conv1(x)  # (batch, 6, 8, 8)
        x = F.relu(x)
        x = self.pool1(x)  # (batch, 6, 4, 4)

        x = self.conv2(x)  # (batch, 12, 2, 2)
        x = F.relu(x)
        x = self.pool2(x)  # (batch, 12, 1, 1)

        x = x.view(x.size(0), -1)  # flatten => (batch, 12)
        x = self.fc(x)            # => (batch, 4)
        return x

# Instantiate a model instance.
model = MultiLetterCNN(num_classes=4)
model.eval()  # set model to eval mode for consistency
model

MultiLetterCNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=12, out_features=4, bias=True)
)

## **2. Model Summary (Text-Based)**

The **torchsummary** (or **torch-summary**) package prints a concise table with the following columns:
- Layer name
- Output shape (N, C, H, W)
- Number of parameters

If you haven't installed torchsummary, run:
```bash
!pip install torchsummary
```

Then we can do the following to get a summary of our model:


In [4]:
# If needed: !pip install torchsummary

from torchsummary import summary

# We expect input of shape (1, 10, 10) for our single-channel 10x10 images.
summary(model, input_size=(1, 10, 10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 6, 8, 8]              60
         MaxPool2d-2              [-1, 6, 4, 4]               0
            Conv2d-3             [-1, 12, 2, 2]             660
         MaxPool2d-4             [-1, 12, 1, 1]               0
            Linear-5                    [-1, 4]              52
Total params: 772
Trainable params: 772
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


You should see a table indicating each layer (Conv2d, MaxPool2d, etc.), its output shape, and the total parameters. This allows a quick textual overview of how the data flows through the network.

---

## **3. Graph Visualization with *torchviz***

Next, we'll demonstrate a detailed visualization of our network as a graph, showing how each operation is connected. We'll:
1. Pass a **dummy input** (random tensor) into the model to capture the computational graph.
2. Use **`make_dot`** from `torchviz` to visualize the graph.

**Note**: `torchviz` uses [Graphviz](https://graphviz.org/) under the hood. Make sure you have Graphviz installed on your machine as well (e.g., `sudo apt-get install graphviz` on Ubuntu or use an installer on Windows).

In [5]:
# If needed: !pip install torchviz

dummy_input = torch.randn(1, 1, 10, 10)  # (batch_size=1, channels=1, height=10, width=10)
output = model(dummy_input)

# Create a graphical representation of the model.
# We map each parameter name => parameter.
dot = make_dot(
    output,
    params=dict(model.named_parameters()),  # This adds parameter nodes
    show_attrs=False,
    show_saved=False
)

# You can render the graph to a file.
# For instance, to create a PNG:
# dot.render("MultiLetterCNN_Graph", format="png")

# Instead, let's display the raw graph in the notebook.
# NOTE: Some notebook environments will display an interactive graph
# but if it doesn't show inline, you might have to open the file.
dot.render("MultiLetterCNN_Graph", format="png")


'MultiLetterCNN_Graph.png'

In [6]:
# !sudo port install graphviz
!brew install graphviz



To reinstall 12.2.1, run:
  brew reinstall graphviz


Depending on your notebook environment, the graph may appear inline or you might see a link to the `.svg` or `.pdf` file. Clicking on it should open the full network graph. If you have Graphviz properly installed, you can also run:
```python
dot.render("MultiLetterCNN_Graph", format="png")
```
and then check your current directory for **`MultiLetterCNN_Graph.png`**, which you can open to see a comprehensive flow diagram of how data travels through your CNN.

### **Graph Explanation**
- **Blue nodes** typically represent **model parameters** (weights, biases, etc.).
- **Orange nodes** represent operations (like convolution, ReLU, pooling, linear, etc.).
- **White nodes** represent **tensors** flowing between operations.

By examining the graph, you get a detailed, step-by-step visualization of the entire forward pass.

---

## **4. Putting It All Together**

Here's a concise recap:
1. **Model Summary** gives a quick textual overview of shapes and params.
2. **Graph Visualization** (torchviz) provides an in-depth, graphical breakdown of every operation in the network.

Together, these tools give a **very comprehensive** understanding of your CNN—both at a high-level (layer-by-layer) and a low-level (tensor flow through the network) perspective.

___
**Quest on!**